In [1]:
import warnings

import pandas as pd

from MLWrappers import SklearnBlackBox
from PrivacyAttacks import MiaPrivacyAttack, LabelOnlyPrivacyAttack, AloaPrivacyAttack
from MLPrivacyEvaluator import PrivacyEvaluator

warnings.simplefilter("ignore", UserWarning)

In [2]:
DS_NAME = 'gaussian'
DATA_FOLDER = f'./data/{DS_NAME}'

# we load the target black box model using our wrapper
target = SklearnBlackBox(f'./models/rf_{DS_NAME}.pkl')

# We load the data used to train, test of the model, as well as the shadow data
train_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_train_set.csv', skipinitialspace=True)[:1000]
test_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_test_set.csv', skipinitialspace=True)[:1000]
shadow_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_shadow_set.csv', skipinitialspace=True)[:1000]

In [3]:
shadow_tree_params = {'min_samples_leaf': 100,
                      'max_depth': 10}
shadow_forest_params = {'n_estimators': 17,
                        'max_depth': 5}

In [4]:
# We initialise the attacks, with the desired parameters for each
mia = MiaPrivacyAttack(target,
                       n_shadow_models=5)
label_only = LabelOnlyPrivacyAttack(target,
                                    n_shadow_models=1,
                                    shadow_model_type='dt',
                                    shadow_model_params=shadow_tree_params)
aloa = AloaPrivacyAttack(target,
                         n_shadow_models=1,
                         shadow_model_type='rf',
                         shadow_model_params=shadow_forest_params,
                         n_noise_samples_fit=200,
                         shadow_test_size=0.3,
                         undersample_attack_dataset=True)
attacks = [mia, label_only, aloa]

In [5]:
# We initialise the PrivacyEvaluator object
# We pass the target model and the attacks we want to use
evaluator = PrivacyEvaluator(target, attacks)

In [6]:
# We use the fit() method to execute the attacks, starting from the shadow data
evaluator.fit(shadow_set)

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_le

100%|██████████| 1000/1000 [00:21<00:00, 45.78it/s]


Each shadow model uses 100.000 % of the data
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 17, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


 45%|████▍     | 269/600 [00:06<00:07, 43.95it/s]


KeyboardInterrupt: 

In [ ]:
# Then we can obtain the performances using the report() method
results = evaluator.report(train_set, test_set)
print(results)


100%|██████████| 2000/2000 [00:40<00:00, 49.80it/s]

{'mia_attack': {'classification_report': {'IN': {'precision': 0.5196599362380446, 'recall': 0.978, 'f1-score': 0.6786953504510757, 'support': 1000}, 'OUT': {'precision': 0.8135593220338984, 'recall': 0.096, 'f1-score': 0.17173524150268338, 'support': 1000}, 'accuracy': 0.537, 'macro avg': {'precision': 0.6666096291359715, 'recall': 0.537, 'f1-score': 0.4252152959768795, 'support': 2000}, 'weighted avg': {'precision': 0.6666096291359715, 'recall': 0.537, 'f1-score': 0.4252152959768795, 'support': 2000}}}, 'label_only_attack': {'classification_report': {'IN': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1000}, 'OUT': {'precision': 0.5, 'recall': 1.0, 'f1-score': 0.6666666666666666, 'support': 1000}, 'accuracy': 0.5, 'macro avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 2000}, 'weighted avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 2000}}}, 'aloa_attack': {'classification_report': {'IN': {'precisio

In [ ]:
results['mia_attack']

{'classification_report': {'IN': {'precision': 0.5196599362380446,
   'recall': 0.978,
   'f1-score': 0.6786953504510757,
   'support': 1000},
  'OUT': {'precision': 0.8135593220338984,
   'recall': 0.096,
   'f1-score': 0.17173524150268338,
   'support': 1000},
  'accuracy': 0.537,
  'macro avg': {'precision': 0.6666096291359715,
   'recall': 0.537,
   'f1-score': 0.4252152959768795,
   'support': 2000},
  'weighted avg': {'precision': 0.6666096291359715,
   'recall': 0.537,
   'f1-score': 0.4252152959768795,
   'support': 2000}}}

In [ ]:
results['aloa_attack']['classification_report']

{'IN': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1000},
 'OUT': {'precision': 0.5,
  'recall': 1.0,
  'f1-score': 0.6666666666666666,
  'support': 1000},
 'accuracy': 0.5,
 'macro avg': {'precision': 0.25,
  'recall': 0.5,
  'f1-score': 0.3333333333333333,
  'support': 2000},
 'weighted avg': {'precision': 0.25,
  'recall': 0.5,
  'f1-score': 0.3333333333333333,
  'support': 2000}}

In [ ]:
aloa.attack_model.threshold

0.62